## Loading required libraries

In [2]:
import cv2
import urllib
import numpy as np
from os import path
# from skimage import io
# from google.colab.patches import cv2_imshow #cv2.imshow is disabled in colab

## Utility functions

### Read img from URL  - url2img(url_of_img, mode=-1)

In [2]:
BASE_REPO = r"https://raw.githubusercontent.com/opencv/opencv/master/samples/data"

def url2img(url_of_img, mode=-1):
    with urllib.request.urlopen(url_of_img) as url:
        img_array = np.array(bytearray(url.read()), dtype=np.uint8)
    img = cv2.imdecode(img_array, mode)
    # OpenCV uses BGR format
    # img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    return img

### Show img - show_img(img, time=5000, fname='Frame')

In [3]:
def show_img(img, time=5000, fname='Frame_name', img_name='written_img.jpg'):
    try:
        cv2.imshow(fname, img)
        # for 64-bit system 0xFF
        k = cv2.waitKey(0) & 0xFF
        if k == 27:  # Esc key, close image window
            cv2.destroyAllWindows()
        elif k == ord('s'):  # s to save image, and exit
            cv2.imwrite(img_name, img)
    except Exception:
        print('Something is wrong!')
    finally:
        cv2.destroyAllWindows()

## Image Operations

### Read image from URL

In [4]:
# unchanged image, with alpha layer too, default mode=-1, unchanged
img_path = BASE_REPO + r'/lena.jpg'
lena_unchanged = url2img(img_path)
show_img(lena_unchanged)

In [5]:
# grayscale
lena_gray = url2img(img_path, 0)
show_img(lena_gray)

In [6]:
# BGR format
lena_color = url2img(img_path, 1)
show_img(lena_color)

### Reading/Writing local image

In [7]:
cv2.imwrite('.\data\lena_gray.jpg', lena_gray)

True

In [8]:
show_img(cv2.imread('.\data\lena_gray.jpg'))  ## read and display image

### Drawing Shapes on frames

In [9]:
img = cv2.imread('.\data\lena.jpg')
img = cv2.line(img, (0, 0), (255, 255), color=(255, 0, 0), thickness=2)
img = cv2.arrowedLine(img, (255, 255), (400, 255), color=(0, 0, 0), thickness=3)
img = cv2.rectangle(img, (384, 0), (510, 128), color=(0, 200, 0), thickness=1)
img = cv2.circle(img, (447, 363), 63, color=(0, 200, 0), thickness=-1)
font=cv2.FONT_HERSHEY_SIMPLEX
img = cv2.putText(img, text='OpenCV', org=(10, 500), 
                  fontFace=font, fontScale=4, 
                  color=(200, 200, 200), thickness=2)
cv2.imshow('image', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Image Operations

### shape, size, dtype, split, merge

In [10]:
img = cv2.imread('.\data\messi5.jpg')

print(f"img.shape - {img.shape}")  # (row, col, channel)
print(f"img.size - {img.size}")  # total pixels
print(f"img.dtype - {img.dtype}")

b, g, r = cv2.split(img)  # split image channel wise
# img2 = cv2.merge((b, g, r))
img2 = cv2.merge((r, g, b))

cv2.imshow('image', img)
cv2.imshow('image2', img2)
cv2.waitKey(0)
cv2.destroyAllWindows()

img.shape - (342, 548, 3)
img.size - 562248
img.dtype - uint8


### copy ROI (region of interest)

In [11]:
img = cv2.imread('.\data\messi5.jpg')

ball = img[280:340, 330:390]  # copy ball 
img[273:333, 100:160] = ball  # paste ball

cv2.imshow('image', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Add 2 images, resize

In [12]:
# https://docs.opencv.org/2.4/modules/core/doc/operations_on_arrays.html
img = cv2.imread('.\data\messi5.jpg')
img2 = cv2.imread('.\data\opencv-logo.png')

img = cv2.resize(img, (512, 512))
img2 = cv2.resize(img2, (512, 512))

new_img = cv2.add(img, img2)
new_img2 = cv2.addWeighted(img, .9, img2, .1, 0)

cv2.imshow('image', new_img)
cv2.imshow('image2', new_img2)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Bitwise operations

In [13]:
img1 = np.zeros((250, 500, 3), np.uint8)
img1 = cv2.rectangle(img1, (200, 0), (300, 100), (255, 255, 255), -1)
img2 = np.zeros((250, 500, 3), np.uint8)
img2 = cv2.rectangle(img2, (250, 0), (500, 250), (255, 255, 255), -1)

cv2.imshow('image', img1)
cv2.imshow('image2', img2)
cv2.imshow('AND', cv2.bitwise_and(img1, img2))
cv2.imshow('OR', cv2.bitwise_or(img1, img2))
cv2.imshow('XOR', cv2.bitwise_xor(img1, img2))
# cv2.imshow('NOT', cv2.bitwise_and(img1))
cv2.waitKey(0)
cv2.destroyAllWindows()

## Video Operations

### Capturing from webcam/video file

In [14]:
video_file = r".\data\video.webm"
# create capture object
cap = cv2.VideoCapture(video_file)  # 0 index of camera,
# other options : -1, 1, 2, etc.. for multiple devices

print(f"Is Valid cap? - {cap.isOpened()}")

while cap.isOpened():  # if index is wrong or, file doesn't exist, nothing happens
    ret, frame = cap.read()
    # convert to gray
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    cv2.imshow('frame_name', frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

print(f"frame width - {cap.get(cv2.CAP_PROP_FRAME_WIDTH)}")

cap.release()  # release resources
cv2.destroyAllWindows()


Is Valid cap? - True
frame width - 960.0


### Writing a video file

In [15]:
# capturing a video and saving it
# https://www.fourcc.org/codecs.php
fourcc = cv2.VideoWriter_fourcc(*'XVID')
# cv2.VideoWriter(filename, fourcc, fpd, dims)
out = cv2.VideoWriter('.\data\out.avi', fourcc, 20.0, (640, 480))

cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    out.write(frame)
    cv2.imshow('frame_name', frame)  # colab
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
        
cap.release()
out.release()
cv2.destroyAllWindows()

### Setting camera params

In [16]:
video_file = r".\data\video.webm"

cap = cv2.VideoCapture(0)

print(f"Is Valid cap? - {cap.isOpened()}")
print(f"frame width - {cap.get(cv2.CAP_PROP_FRAME_WIDTH)}")
print(f"frame height - {cap.get(cv2.CAP_PROP_FRAME_HEIGHT)}")

# every property is associated with some number - 3:width, 4:height
# also it will maintain ratio, and only select from available resolutions
cap.set(3, 200)
cap.set(4, 200)

print(f"frame width - {cap.get(3)}")
print(f"frame height - {cap.get(4)}")

while cap.isOpened():
    ret, frame = cap.read()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    cv2.imshow('frame_name', frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
cap.release()
cv2.destroyAllWindows()


Is Valid cap? - True
frame width - 640.0
frame height - 480.0
frame width - 320.0
frame height - 240.0


### Adding text to frames

In [17]:
from datetime import datetime as dt

cap = cv2.VideoCapture(video_file)
print(f"frame width - {cap.get(cv2.CAP_PROP_FRAME_WIDTH)}")
print(f"frame height - {cap.get(cv2.CAP_PROP_FRAME_HEIGHT)}")

while cap.isOpened():
    ret, frame = cap.read()
    font = cv2.FONT_HERSHEY_SIMPLEX
    text = f"W:{cap.get(3)} H:{cap.get(4)}DT: {dt.now()}"
    frame = cv2.putText(frame, text=text, 
                        org=(10, 50), 
                        fontFace=font, 
                        fontScale=1, 
                        color=(0, 0, 0), 
                        thickness=2,
                        lineType=cv2.LINE_AA)
    cv2.imshow('frame_name', frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
cap.release()
cv2.destroyAllWindows()


frame width - 960.0
frame height - 540.0


## Mouse events

In [18]:
# get a lis tof events
events = [event for event in dir(cv2) if 'EVENT' in event]
events

['EVENT_FLAG_ALTKEY',
 'EVENT_FLAG_CTRLKEY',
 'EVENT_FLAG_LBUTTON',
 'EVENT_FLAG_MBUTTON',
 'EVENT_FLAG_RBUTTON',
 'EVENT_FLAG_SHIFTKEY',
 'EVENT_LBUTTONDBLCLK',
 'EVENT_LBUTTONDOWN',
 'EVENT_LBUTTONUP',
 'EVENT_MBUTTONDBLCLK',
 'EVENT_MBUTTONDOWN',
 'EVENT_MBUTTONUP',
 'EVENT_MOUSEHWHEEL',
 'EVENT_MOUSEMOVE',
 'EVENT_MOUSEWHEEL',
 'EVENT_RBUTTONDBLCLK',
 'EVENT_RBUTTONDOWN',
 'EVENT_RBUTTONUP']

### Mouse click example

In [19]:
# create mouse callback function
def click_event(event, x, y, flags, param):
    # on double left mouse click, show coordinates
    if event == cv2.EVENT_LBUTTONDBLCLK:
        font = cv2.FONT_HERSHEY_SIMPLEX
        text = f"{x}, {y}"
        print(text)
        cv2.putText(img, text=text, org=(x, y), 
                    fontFace=font, fontScale=.5, 
                    color=(255, 255, 0), thickness=2,)
        cv2.imshow('frame_name', img)

img = np.zeros((512, 512, 3), np.uint8)
cv2.imshow('frame_name', img)

cv2.setMouseCallback('frame_name', click_event)

cv2.waitKey(0)
cv2.destroyAllWindows()

137, 124
159, 230


### draw line via mouse

In [20]:
# create mouse callback function
def click_event(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDBLCLK:
        cv2.circle(img, (x, y), radius=3, color=(0, 0, 255), thickness=-1)
        points.append((x, y))
        if len(points) >= 2:
            cv2.line(img, points[-1], points[-2], (255, 0, 0), 5)
        cv2.imshow('frame_name', img)

img = np.zeros((512, 512, 3), np.uint8)
cv2.imshow('frame_name', img)
points = []
cv2.setMouseCallback('frame_name', click_event)

cv2.waitKey(0)
cv2.destroyAllWindows()

## Trackbar

### change color

In [3]:
img = np.zeros((300, 512, 3), np.uint8)

win_name = 'image'
cv2.namedWindow(win_name)  # create named window

# callback function
def do_something(x):
    return x

# cv2.createTrackbar(trackbarName='B', 
#                    windowName='image', # named window name
#                    value=0,  # set value
#                    count=255, # max val
#                    onChange=do_something, # callback function
#                   )
cv2.createTrackbar('B', win_name, 0, 255, do_something)
cv2.createTrackbar('G', win_name, 200, 255, do_something)
cv2.createTrackbar('R', win_name, 0, 255, do_something)
cv2.createTrackbar('s', win_name, 0, 1, do_something)

while True:
    cv2.imshow(win_name, img)
    k = cv2.waitKey(1) & 0xFF
    if k == 27:
        break
    b = cv2.getTrackbarPos('B', win_name)
    g = cv2.getTrackbarPos('G', win_name)
    r = cv2.getTrackbarPos('R', win_name)
    
    
    if cv2.getTrackbarPos('s', win_name):
        img[:] = [b, g, r]

cv2.destroyAllWindows()

### another example

In [4]:
win_name = 'image'
cv2.namedWindow(win_name)

cv2.createTrackbar('num', win_name, 30, 400, do_something)
cv2.createTrackbar('Color/Gray', win_name, 0, 1, do_something)

while True:
    img = cv2.imread('./data/lena.jpg')
    pos = cv2.getTrackbarPos('num', win_name)
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(img, str(pos), (50, 150), font, 4, (0, 0, 0))
    
    
    k = cv2.waitKey(1) & 0xFF
    if k ==27:
        break
        
    if cv2.getTrackbarPos('Color/Gray', win_name):
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    cv2.imshow(win_name, img)

cv2.destroyAllWindows()    

## HSV (hue, saturation, value)